In [ ]:
from nltk.tokenize import sent_tokenize
import re
import torch

In [ ]:
"""Helper functions for tokenizing"""
def Read_Corpus(path):
    corpus_file = open(path, 'r')
    lines = corpus_file.readlines()
    text = ""
    for line in lines:
        striped_line = line.strip()
        if striped_line != '':
            text = text + " " + striped_line
    return text


def substitue_tokens(sentences):
    sub_sentences = []
    for sentence in sentences:
        sub_sen = sentence.lower()
        sub_sen = re.sub("#[a-zA-Z0-9_]+", "<HASHTAG>", sub_sen)
        sub_sen = re.sub("@[a-zA-Z0-9_]+", "<MENTION>", sub_sen)
        sub_sen = re.sub("https?://[a-zA-Z0-9_./]+", "<URL>", sub_sen)
        sub_sentences.append(sub_sen)
    return sub_sentences


def split_corpus_into_sentences(corpus):
    sentences = sent_tokenize(corpus)
    return sentences


def tokenize_sentence(sen):
    # Handles words 
    word_wise = sen.split()

    # Handeling punctuations
    tokenized_sen = []
    for word in word_wise:
        if(is_punc(word) == True):
            tokenized_sen.append(word)
        else:
            var_word = word
            while(len(var_word) != 0 and starting_punc(var_word) == True):
                tokenized_sen.append(word[0])
                var_word = var_word[1:]
            
            end_puncs = []
            while(len(var_word) != 0 and ending_punc(var_word) == True):
                end_puncs = [var_word[-1]] + end_puncs
                var_word = var_word[:-1]
            
            tokenized_sen.append(var_word)
            tokenized_sen += end_puncs
    
    return tokenized_sen


# Does the given word have punctation at the end
def ending_punc(word):
    if(word[-1] == ',') or (word[-1] == ':') or (word[-1] == ';') or (word[-1] == '"') or (word[-1] == ')') or (word[-1] == '}') or (word[-1] == ']') or (word[-1] == '.') or (word[-1] == '?') or (word[-1] == '!'):
        return True
    else:
        return False
    

# Does the given word have punctation at the start
def starting_punc(word):
    if(word[0] == '"') or (word[0] == '(') or (word[0] == '{') or (word[0] == '['):
        return True
    else:
        return False
    

# Is the given word a punctation
def is_punc(word):
    if(len(word) == 1 and (ending_punc(word) or starting_punc(word))):
        return True
    else:
        return False
    

def tokenize_corpus(path):
    corpus = Read_Corpus(path)
    sentences = sent_tokenize(corpus)
    url_metions = substitue_tokens(sentences)
    sentence_tokenized = []
    for sentence in url_metions:
        tokenized_sen = tokenize_sentence(sentence)
        sentence_tokenized.append(tokenized_sen)
    return sentence_tokenized


In [ ]:
path = "Pride and Prejudice - Jane Austen.txt"
tokenized_corpus = tokenize_corpus(path)

In [ ]:
def unigram_from_token_corpus(tokens_sen):
    Counts = {}
    for tokens in tokens_sen:
        for token in tokens:
            if token in Counts.keys():
                Counts[token] += 1
            else:
                Counts[token] = 1
    return Counts


def vocab_index(unigram_counts):
    vocab_index = {}
    index = 0
    for key in unigram_counts.keys():
        if(unigram_counts[key] > 1):
            vocab_index[key] = index
            index += 1
    vocab_index["<UNK>"] = index       
    return vocab_index


def get_index(word, vocab_index):
    if word in vocab_index.keys():
        return vocab_index[word]
    else:
        return vocab_index["<UNK>"]
    

unigram_counts = unigram_from_token_corpus(tokenized_corpus)
print(sum(unigram_counts.values()))
vocab_idx = vocab_index(unigram_counts)

In [ ]:
print("Vocab size: ", len(vocab_idx))
#print(vocab_idx.values())
assert(list(vocab_idx.values()) == [i for i in range(len(vocab_idx))])
#print([get_index(key, vocab_idx) for key in vocab_idx.keys()])
assert([get_index(key, vocab_idx) for key in vocab_idx.keys()] == [i for i in range(len(vocab_idx))])
#print(vocab_idx['?'])

In [9]:
def get_data(tokenized_corpus, vocab_idx, batch_size):
    data = []                                                   
    for tokenized_sentence in tokenized_corpus:            
            tokens = [get_index(token, vocab_idx) for token in tokenized_sentence] 
            data.extend(tokens)              
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size 
    data = data[:num_batches * batch_size]                       
    data = data.view(batch_size, num_batches)          
    return data

def get_batch(data, seq_len, num_batches, idx):
    assert(idx + seq_len + 1 < num_batches)
    src = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]             
    return src, target

In [11]:
data = get_data(tokenized_corpus, vocab_idx, 32)
for i in range(100):
    pass
    #print(get_batch(data, 10, data.shape[1], i))